# How to fine-tune chat models

## 데이터 준비

In [ ]:
# make sure to use the latest version of the openai python package
!pip install --upgrade openai 

In [1]:
from dotenv import load_dotenv
import os

# .env 파일 활성화
load_dotenv('.')
API_KEY = os.getenv('OPENAI_API_KEY')


In [2]:
from openai import OpenAI


In [20]:
import os
import openai
import requests
import pprint
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import base64
import io
import pandas as pd

client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY", '*'))

# openai.api_key = os.getenv('OPENAI_API_KEY')

In [2]:
# 이미지를 base64로 인코딩하고 PIL 이미지 객체를 반환하는 함수
def load_and_encode_images(image_sources):
    encoded_images = []
    pil_images = []
    for source in image_sources:
        if source.startswith('http'):  # URL인 경우
            response = requests.get(source)
            image_data = response.content
        else:  # 파일 경로인 경우
            with open(source, "rb") as image_file:
                image_data = image_file.read()
 
        pil_images.append(Image.open(io.BytesIO(image_data)))
        encoded_images.append(base64.b64encode(image_data).decode('utf-8'))
    return encoded_images, pil_images


In [3]:
# 응답결과와 이미지를 출력하기 위한 함수
def display_response(pil_images, response_text):
    # 이미지 로딩 및 서브플롯 생성
    fig, axes = plt.subplots(nrows=1, ncols=len(pil_images), figsize=(5 * len(pil_images), 5))
    if len(pil_images) == 1:  # 하나의 이미지인 경우
        axes = [axes]
 
    # 이미지들 표시
    for i, img in enumerate(pil_images):
        axes[i].imshow(img)
        axes[i].axis('off')  # 축 정보 숨기기
        axes[i].set_title(f'Image #{i+1}')
 
    # 전체 플롯 표시
    plt.show()
 
    print(response_text)

In [4]:
# 이미지 경로 또는 URL과 프롬프트를 처리하는 함수
def process_and_display_images(image_sources, prompt):
    # 이미지 로드, base64 인코딩 및 PIL 이미지 객체 생성
    base64_images, pil_images = load_and_encode_images(image_sources)
 
    # OpenAI에 요청 보내기
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt}
            ] + [{"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}} for base64_image in base64_images]
        }
    ]
 
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=1000
    )
 
    response_text = response.choices[0].message.content
 
    # 응답과 이미지 표시
    display_response(pil_images, response.choices[0].message.content)
 
    return response_text

## Data preparation

```
[{'role': 'system',
  'content': '너는 입력받은 한국어를 쉬운 한국어로 변환해주는 도우미야. 다음 조건들을 모두 충족하는 내용으로 변환해서 알려줘. \n 1. 입력받은 한국어 문장을 이해하기 쉬운 한국어 문장으로 변환해줘. 2. 문장은 간결하게, 한 문장이 길어지면 두 문장으로 나눠서 변환해줘. 3. 꾸미는 말 빼고, 이어진 문장은 두 개의 문장으로 변환해줘. 4. 주어를 중심으로 알기 쉽게 변환해줘. 5. 최대한 능동형 문장으로, 서술식의 구어체(-합니다, -입니다)로 변환해줘. 6. 추상적 표현과 비유는 자제하도록 해. 7. 이중부정 문장은 이해하기 쉬운 문장으로 바꿔. 8. 한 문장에 한 줄씩 적어야해. 9. 대화문은 문장 전후로 한 줄 띄어줘. 10. 단어는 일상생활에서 자주 쓰는, 가능한 짧고 이해하기 쉬운 단어로 사용하도록 해. 11. 한자어나 외국어를 풀어서 쉬운 말로 변환해. 12. 약어가 있으면 다음 문장에 설명을 추가해.'},

 {'role': 'user',
  'content': '변환 전 텍스트'},

 {'role': 'assistant',
  'content': '변환 후 텍스트'}]
```

훈련 과정에서 이 대화는 나누어질 것이며, 최종 항목은 모델이 생성할 `completion`이 되고 나머지 `messages`는 프롬프트로 작용합니다. 모델이 다중 턴 대화에서 작동하도록 할 경우, 대화가 확장될 때 성능이 저하되지 않도록 대표적인 예제를 제공해야 합니다.

현재 각 훈련 예제에는 4096 토큰 제한이 있으므로, 이보다 긴 경우에는 4096 토큰에서 잘리게 됩니다.

In [57]:
reply = []
reply.append(rere)

In [54]:
user = []
user.append(s)

In [317]:
training_data = []

# 시스템 메세지 기입
system_message = "너는 입력받은 한국어를 쉬운 한국어로 변환해주는 도우미야. 다음 조건들을 모두 충족하는 내용으로 변환해서 알려줘. 1. 입력받은 한국어 문장을 이해하기 쉬운 한국어 문장으로 변환해줘. 2. 문장은 간결하게, 한 문장이 길어지면 두 문장으로 나눠서 변환해줘. 3. 꾸미는 말 빼고, 이어진 문장은 두 개의 문장으로 변환해줘. 4. 주어를 중심으로 알기 쉽게 변환해줘. 5. 최대한 능동형 문장으로, 서술식의 구어체(-합니다, -입니다)로 변환해줘. 6. 추상적 표현과 비유는 자제하도록 해. 7. 이중부정 문장은 이해하기 쉬운 문장으로 바꿔. 8. 한 문장에 한 줄씩 적어야해. 9. 대화문은 문장 전후로 한 줄 띄어줘. 10. 단어는 일상생활에서 자주 쓰는, 가능한 짧고 이해하기 쉬운 단어로 사용하도록 해. 11. 한자어나 외국어를 풀어서 쉬운 말로 변환해. 12. 약어가 있으면 다음 문장에 설명을 추가해."

# 입력할 메세지를 데이터프레임 컬럼에서 가져옴.
def create_user_message(df):
    # return f"""Title: {row['title']}\n\nIngredients: {row['ingredients']}\n\nGeneric ingredients: """
    return f"""{df['']}"""


# 전체적인 딕셔너리 만들기
def prepare_example_conversation(df):
    messages = []
    messages.append({"role": "system", "content": system_message})

    # user_message = create_user_message(df)
    messages.append({"role": "user", "content": df['user']})

    messages.append({"role": "assistant", "content": df["reply"]})

    return {"messages": messages}

# pprint(prepare_example_conversation(df.iloc[0]))

In [60]:
# use the first 100 rows of the dataset for training
training_df = df.loc[0:9]

# apply the prepare_example_conversation function to each row of the training_df
training_data = training_df.apply(prepare_example_conversation, axis=1).tolist()

for example in training_data[:5]:
    print(example)

{'messages': [{'role': 'system', 'content': '너는 입력받은 한국어를 쉬운 한국어로 변환해주는 도우미야. 다음 조건들을 모두 충족하는 내용으로 변환해서 알려줘. 1. 입력받은 한국어 문장을 이해하기 쉬운 한국어 문장으로 변환해줘. 2. 문장은 간결하게, 한 문장이 길어지면 두 문장으로 나눠서 변환해줘. 3. 꾸미는 말 빼고, 이어진 문장은 두 개의 문장으로 변환해줘. 4. 주어를 중심으로 알기 쉽게 변환해줘. 5. 최대한 능동형 문장으로, 서술식의 구어체(-합니다, -입니다)로 변환해줘. 6. 추상적 표현과 비유는 자제하도록 해. 7. 이중부정 문장은 이해하기 쉬운 문장으로 바꿔. 8. 한 문장에 한 줄씩 적어야해. 9. 대화문은 문장 전후로 한 줄 띄어줘. 10. 단어는 일상생활에서 자주 쓰는, 가능한 짧고 이해하기 쉬운 단어로 사용하도록 해. 11. 한자어나 외국어를 풀어서 쉬운 말로 변환해. 12. 약어가 있으면 다음 문장에 설명을 추가해.'}, {'role': 'user', 'content': '<개학 날 맡은 업무> \n 오늘은 2학년이 되는 새 학기 첫날입니다.\n드르륵 문이 열리는 소리가 나더니 2학년 2반 교실 안으로 누군가 들어섰습니다. 조기준 선생님과 아이들의 눈이 한꺼번에 문 쪽으로 쏠렸습니다.\n잿빛 코트를 입은 아주머니 한 분이 조심스럽게 고개를 숙여 인사했습니다.\n“선생님, 안녕하세요? 이 반에 배정된 민영택이 엄마입니다.”\n“아, 그러세요? 그런데 영택이는요?”\n“여기 있습니다.”\n아주머니 뒤로 영택이라는 아이가 들어왔습니다. 양 팔꿈치에 하얀 알루미늄으로 된 목발을 끼우고 걷는 장애아였습니다. 태어나서 햇빛이라고는 한 번도 받아 본 적이 없는 아이처럼 얼굴이 하얬습니다.\n“여기 앉아라.”\n선생님이 비어 있는 맨 앞자리를 가리켰습니다. 영택이는 비틀거리며 걸어와 목발을 바닥에 내려 놓고 걸상에 털썩 앉았습니다.\n그새 아주머니와 선생님은 복도로 나가 이야기를 나누고 있었습니다\n“야. 쟤 

In [61]:
import json

In [133]:
sample['messages'][1]['content']

'평소 화재에 대한 관심을 갖고, 화재 발생 시 행동 요령을 숙지해야 합니다. 화재가 발생하면 침착하게 행동합니다.'

In [152]:
for num, sample in enumerate(origin):
    origin_system = origin[num]['messages'][0]['content']
    origin_user = origin[num]['messages'][1]['content']
    origin_ass = origin[num]['messages'][2]['content']
    if '각주 처리' in origin_system:
        origin_system = '어려운 어휘를 쉬운 어휘로 변경하기 어려운 경우, 다음 문장에 쉬운 단어로 설명하기 혹은 괄호 안에 동일한 의미의 쉬운 어휘 제시하기.'
        if '(' in origin_user:
            a = origin_ass
            origin_ass = origin_user
            origin_user = a
        print(origin_system, origin_user, origin_ass)

            
write_jsonl(origin, "data/exdata3.jsonl")


어려운 어휘를 쉬운 어휘로 변경하기 어려운 경우, 다음 문장에 쉬운 단어로 설명하기 혹은 괄호 안에 동일한 의미의 쉬운 어휘 제시하기. 그는 항상 자신의 믿음을 지킵니다. 그는 항상 소신(자신의 믿음)을 지킨다.
어려운 어휘를 쉬운 어휘로 변경하기 어려운 경우, 다음 문장에 쉬운 단어로 설명하기 혹은 괄호 안에 동일한 의미의 쉬운 어휘 제시하기. 그는 타고난 리더십을 발휘하여 팀을 이끌었다. 그는 남을 잘 이끌어 팀을 이끌었습니다.(리더십은 남을 잘 이끄는 능력입니다.)
어려운 어휘를 쉬운 어휘로 변경하기 어려운 경우, 다음 문장에 쉬운 단어로 설명하기 혹은 괄호 안에 동일한 의미의 쉬운 어휘 제시하기. 그녀는 뛰어난 통찰력을 보여주었다. 그녀는 잘 이해하고 판단하는 능력(통찰력)을 보여주었습니다.
어려운 어휘를 쉬운 어휘로 변경하기 어려운 경우, 다음 문장에 쉬운 단어로 설명하기 혹은 괄호 안에 동일한 의미의 쉬운 어휘 제시하기. 그녀는 항상 적극적입니다. 그녀는 매사에 진취적(적극적)이다.
어려운 어휘를 쉬운 어휘로 변경하기 어려운 경우, 다음 문장에 쉬운 단어로 설명하기 혹은 괄호 안에 동일한 의미의 쉬운 어휘 제시하기. 그는 사회적 의무를 다하기 위해 노력한다. 그는 사회적(사회와 관련된) 의무를 다하려고 노력합니다.
어려운 어휘를 쉬운 어휘로 변경하기 어려운 경우, 다음 문장에 쉬운 단어로 설명하기 혹은 괄호 안에 동일한 의미의 쉬운 어휘 제시하기. 그녀는 매사에 진취적이다. 그녀는 항상 적극적입니다.
어려운 어휘를 쉬운 어휘로 변경하기 어려운 경우, 다음 문장에 쉬운 단어로 설명하기 혹은 괄호 안에 동일한 의미의 쉬운 어휘 제시하기. 이 프로젝트는 매우 포괄적이다. 이 프로젝트는 매우 전반적(포괄적)입니다.


In [153]:
instruct = []
for sam in origin:
    instruct.append(sam['messages'][0]['content'])
list(set(instruct))

['각 문단의 길이는 비슷하게 유지한다.',
 '행정용어와 법률용어의 사용은 주의한다. 사용해야 한다면, 쉬운 어휘로 그 의미를 부연설명을 해야 한다.',
 '새로운 내용은 기존의 관련 정보와 연결시킨다.',
 '등장인물의 수가 많지 않고 서로 간에 명확하게 구분되도록 한다. 대사는 발화자의 이름을 기입한다.',
 '특정지역이나 집단에서 통용되는 어휘를 사용 한다면 배경설명을 제공한다.',
 '속어, 은어는 쉬운 말로 풀어서 쓴다.',
 '축약어는 사용하지 않는다. 사용해야 한다면, 축약어는 원 명칭을 괄호 안에 함께 제시한다.',
 '연령에 맞는 관심사와 주제는 유지하되 쉬운 어휘로 표현한다.',
 '어려운 어휘를 사용할 때, 이를 설명하거나 괄호 안에 동일한 의미의 쉬운 어휘 제시하거나 각주 처리를 한다.',
 '한 문장에 여러 개의 동사를 사용하지 않는다.',
 '한 문단 안에서는 한 가지 주제만 다룬다.',
 '축약어는 저자가 아닌 독자의 편의를 위해서만 사용하고, 분명한 이유가 없을 때에는 원 명칭을 그대로 사용 한다.',
 '문장은 가능한 한 짧게 작성한다.',
 '행동의 주체가 분명히 드러나도록 능동문을 사용한다.',
 '쉽고 자주 사용되는 어휘로 표현한다.',
 '어려운 어휘를 사용할 때 어휘 이해를 위한 상황 단서를 제공한다.',
 '특정 지역이나 집단에서 통용되는 어휘를 사용 한다면 배경설명을 제공한다.',
 '단어를 대체하기에 앞서, 글의 의도를 파악할 수 있도록 구체적으로 설명하여 용어 자체보다 개념과 상황을 이해할 수 있도록 해석하여 제시한다.',
 '어려운 어휘보다는 일상적으로 사용하는 어휘로 표현한다.',
 '문단을 구성하는 문장의 수는 가능한 적게 하고 문단의 길이는 짧게 한다.',
 '속어나 은어는 사용하지 않는다.',
 '이야기 구조를 단순화하고 속도감 있게 전개한다.',
 '이야기 구조가 명확하게 드러나게 전개한다. 발생시기가 복잡한 사건이라면, 구체적이고 논리적인 방식으로 수정한다.',
 '청소년이나 성인의 경우 지나치게 낮

In [138]:
origin = read_jsonl("data/exdata2.jsonl")

In [310]:
df

,user,reply
0,<개학 날 맡은 업무>,<개학 날 내가 맡은 일>
1,오늘은 2학년이 되는 새 학기 첫날입니다.,오늘은 2 학년 첫날입니다.
2,드르륵 문이 열리는 소리가 나더니 2학년 2반 교실 안으로 누군가 들어섰습니다. 조...,문이 열리고 2 학년 2 반 교실에 누군가 들어왔습니다. \n조기준 담임 선생님과 ...
3,잿빛 코트를 입은 아주머니 한 분이 조심스럽게 고개를 숙여 인사했습니다.,아주머니는 회색 외투를 입었습니다. \n아주머니가 고개를 숙여 인사했습니다.
4,"“선생님, 안녕하세요? 이 반에 배정된 민영택이 엄마입니다.”","아주머니: 선생님, 안녕하세요. 저는 민영택 엄마입니다."
5,"“아, 그러세요? 그런데 영택이는요?”",선생님: 영택이는 어디 있나요?
6,“여기 있습니다.”,아주머니: 뒤에 있습니다.
7,아주머니 뒤로 영택이라는 아이가 들어왔습니다. 양 팔꿈치에 하얀 알루미늄으로 된 목...,영택이가 들어왔습니다. \n영택이는 걷기 어려운 학생입니다. \n영택이는 양 팔꿈치...
8,“여기 앉아라.” 선생님이 비어 있는 맨 앞자리를 가리켰습니다.,선생님: 여기 앉아라. \n선생님이 맨 앞에 자리를 손으로 가리쳤습니다. \n빈자리...
9,영택이는 비틀거리며 걸어와 목발을 바닥에 내려 놓고 걸상에 털썩 앉았습니다.,영택이는 비틀비틀 걸있습니다. \n목발을 바닥에 놓았습니다. \n의자에 앉았습니다.


In [319]:
validation_df = df #.loc[:11]
validation_data = validation_df.apply(prepare_example_conversation, axis=1).tolist()
validation_data

[{'messages': [{'role': 'system',
    'content': '너는 입력받은 한국어를 쉬운 한국어로 변환해주는 도우미야. 다음 조건들을 모두 충족하는 내용으로 변환해서 알려줘. 1. 입력받은 한국어 문장을 이해하기 쉬운 한국어 문장으로 변환해줘. 2. 문장은 간결하게, 한 문장이 길어지면 두 문장으로 나눠서 변환해줘. 3. 꾸미는 말 빼고, 이어진 문장은 두 개의 문장으로 변환해줘. 4. 주어를 중심으로 알기 쉽게 변환해줘. 5. 최대한 능동형 문장으로, 서술식의 구어체(-합니다, -입니다)로 변환해줘. 6. 추상적 표현과 비유는 자제하도록 해. 7. 이중부정 문장은 이해하기 쉬운 문장으로 바꿔. 8. 한 문장에 한 줄씩 적어야해. 9. 대화문은 문장 전후로 한 줄 띄어줘. 10. 단어는 일상생활에서 자주 쓰는, 가능한 짧고 이해하기 쉬운 단어로 사용하도록 해. 11. 한자어나 외국어를 풀어서 쉬운 말로 변환해. 12. 약어가 있으면 다음 문장에 설명을 추가해.'},
   {'role': 'user', 'content': '<개학 날 맡은 업무> '},
   {'role': 'assistant', 'content': '<개학 날 내가 맡은 일>'}]},
 {'messages': [{'role': 'system',
    'content': '너는 입력받은 한국어를 쉬운 한국어로 변환해주는 도우미야. 다음 조건들을 모두 충족하는 내용으로 변환해서 알려줘. 1. 입력받은 한국어 문장을 이해하기 쉬운 한국어 문장으로 변환해줘. 2. 문장은 간결하게, 한 문장이 길어지면 두 문장으로 나눠서 변환해줘. 3. 꾸미는 말 빼고, 이어진 문장은 두 개의 문장으로 변환해줘. 4. 주어를 중심으로 알기 쉽게 변환해줘. 5. 최대한 능동형 문장으로, 서술식의 구어체(-합니다, -입니다)로 변환해줘. 6. 추상적 표현과 비유는 자제하도록 해. 7. 이중부정 문장은 이해하기 쉬운 문장으로 바꿔. 8. 한 문장에 한 줄씩 적어야해. 9. 대화문은 문장 전후로

We then need to save our data as `.jsonl` files, with each line being one training example conversation.


In [62]:
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict, ensure_ascii = False) + "\n"
            out.write(jout)

def read_jsonl(filename):
    data_list = []
    with open(filename, "r") as file:
        # lines = file.readlines()
        for line in file:
            data = json.loads(line.strip()) 
            data_list.append(data)
    return data_list

In [313]:
# with open('./data/validation_final.jsonl', "w") as out:
#     jout = ''
#     for i in read_jsonl("./data/validation_final.jsonl"):
#         jout = jout + json.dumps(i, ensure_ascii = False) + "\n"
#     jout = jout + json.dumps(example, ensure_ascii = False) + "\n"
#     out.write(jout)

In [320]:
write_jsonl(validation_data, './data/validation_final.jsonl')

In [321]:
exdata = read_jsonl('./data/exdata4.jsonl')
validation_data = read_jsonl('./data/validation_final.jsonl')
validation_data

[{'messages': [{'role': 'system',
    'content': '너는 입력받은 한국어를 쉬운 한국어로 변환해주는 도우미야. 다음 조건들을 모두 충족하는 내용으로 변환해서 알려줘. 1. 입력받은 한국어 문장을 이해하기 쉬운 한국어 문장으로 변환해줘. 2. 문장은 간결하게, 한 문장이 길어지면 두 문장으로 나눠서 변환해줘. 3. 꾸미는 말 빼고, 이어진 문장은 두 개의 문장으로 변환해줘. 4. 주어를 중심으로 알기 쉽게 변환해줘. 5. 최대한 능동형 문장으로, 서술식의 구어체(-합니다, -입니다)로 변환해줘. 6. 추상적 표현과 비유는 자제하도록 해. 7. 이중부정 문장은 이해하기 쉬운 문장으로 바꿔. 8. 한 문장에 한 줄씩 적어야해. 9. 대화문은 문장 전후로 한 줄 띄어줘. 10. 단어는 일상생활에서 자주 쓰는, 가능한 짧고 이해하기 쉬운 단어로 사용하도록 해. 11. 한자어나 외국어를 풀어서 쉬운 말로 변환해. 12. 약어가 있으면 다음 문장에 설명을 추가해.'},
   {'role': 'user', 'content': '<개학 날 맡은 업무> '},
   {'role': 'assistant', 'content': '<개학 날 내가 맡은 일>'}]},
 {'messages': [{'role': 'system',
    'content': '너는 입력받은 한국어를 쉬운 한국어로 변환해주는 도우미야. 다음 조건들을 모두 충족하는 내용으로 변환해서 알려줘. 1. 입력받은 한국어 문장을 이해하기 쉬운 한국어 문장으로 변환해줘. 2. 문장은 간결하게, 한 문장이 길어지면 두 문장으로 나눠서 변환해줘. 3. 꾸미는 말 빼고, 이어진 문장은 두 개의 문장으로 변환해줘. 4. 주어를 중심으로 알기 쉽게 변환해줘. 5. 최대한 능동형 문장으로, 서술식의 구어체(-합니다, -입니다)로 변환해줘. 6. 추상적 표현과 비유는 자제하도록 해. 7. 이중부정 문장은 이해하기 쉬운 문장으로 바꿔. 8. 한 문장에 한 줄씩 적어야해. 9. 대화문은 문장 전후로

In [259]:
len(validation_data[-1]['messages'][1]['content'].split("\n"))
(validation_data[-1]['messages'][1]['content'].split("\n"))
val2 = []
for i in validation_data[-1]['messages'][2]['content'].split("\n"):
    if i != '':
        val2.append(i)

# user = validation_data[-1]['messages'][1]['
# assis = validation_data[-1]['messages'][2]


119

In [267]:
a = ['<개학 날 맡은 업무> ',
 ' 오늘은 2학년이 되는 새 학기 첫날입니다.',
 '드르륵 문이 열리는 소리가 나더니 2학년 2반 교실 안으로 누군가 들어섰습니다. 조기준 선생님과 아이들의 눈이 한꺼번에 문 쪽으로 쏠렸습니다.',
 '잿빛 코트를 입은 아주머니 한 분이 조심스럽게 고개를 숙여 인사했습니다.',
 '“선생님, 안녕하세요? 이 반에 배정된 민영택이 엄마입니다.”',
 '“아, 그러세요? 그런데 영택이는요?”',
 '“여기 있습니다.”',
 '아주머니 뒤로 영택이라는 아이가 들어왔습니다. 양 팔꿈치에 하얀 알루미늄으로 된 목발을 끼우고 걷는 장애아였습니다. 태어나서 햇빛이라고는 한 번도 받아 본 적이 없는 아이처럼 얼굴이 하얬습니다.',
 '“여기 앉아라.” 선생님이 비어 있는 맨 앞자리를 가리켰습니다. ',
 '영택이는 비틀거리며 걸어와 목발을 바닥에 내려 놓고 걸상에 털썩 앉았습니다.',
 '그새 아주머니와 선생님은 복도로 나가 이야기를 나누고 있었습니다',
 '“야. 쟤 1학년 때 5반이었지?” 석우는 옆에 앉은 서경이에게 물었습니다. 서경이는 석우네 이웃 동네에 사는 아이입니다. “응. 공부는 잘하는데 친구가 별로 없대.”',
 '조금 뒤 선생님이 들어왔습니다. 그러고는 아이들을 둘러보며 말했습니다. “혹시 집이 제일교회에서 가까운 사람?” 제일교회는 동네에서 가장 큰 교회입니다. 석우네 집에서 50미터쯤 떨어져 있지요. “저, 저요.” 석우는 누가 손들지 않았나 둘러보다가 조심스럽게 손을 올렸습니다. “그래, 너 이름이 뭐니?” “문석우요.” “참 잘됐다. 학교 오는 길에 새한마트 있지?” “네.” 새한마트는 석우네 동네에서 가장 큰 슈퍼마켓입니다. “새한마트 바로 옆집이 영택이네 집이거든. 석우는 내일부터 학교에 올 때 영택이 집에 들러서 영택이 가방 좀 들고 오너라. 갈 때도 들어다 주고…….” “…….”',
 '석우는 얼떨떨했습니다.',
 '“2학년 한 해 동안만 수고 좀 해라.” “네.” 석우는 선생님의 엄한 말투에 기어들어 가는 목소리로 대답했습니다.',
 '그날 수업은 1학기 시간표 적는 것과 주의 사항 전달로 끝이 났습니다.',
 '“야. 너 어떡하냐?” 선생님이 교실에서 나가자, 서경이가 석우에게 물었습니다. “일 년 내내 재 가방 들어다 주게 생겼잖아.” “할 수 없지, 뭐.” 선생님의 엄한 눈빛을 떠올리며 석우는 쭈뼛쭈뼛 영택이에게 다가갔습니다.',
 '“야. 가방 줘.” 영택이는 석우와 눈을 마주치지 못하고 가방을 힘겹게 들어 올려 석우에게 건넸습니다.']

In [264]:
b = ['<개학 날 내가 맡은 일>',
 '오늘은 2 학년 첫날입니다.',
 '문이 열리고 2 학년 2 반 교실에 누군가 들어왔습니다. \n조기준 담임 선생님과 아이들이 문 쪽을 쳐다봤습니다.',
 '아주머니는 회색 외투를 입었습니다. \n아주머니가 고개를 숙여 인사했습니다.',
 '아주머니: 선생님, 안녕하세요. 저는 민영택 엄마입니다.',
 '선생님: 영택이는 어디 있나요?',
 '아주머니: 뒤에 있습니다.',
 '영택이가 들어왔습니다. \n영택이는 걷기 어려운 학생입니다. \n영택이는 양 팔꿈치에 목발을 끼우고 걷습니다. \n목발은 걷기 어려운 사람이 사용하는 지팡이입니다. \n영택이의 얼굴은 하얀색이었습니다.',
 '선생님: 여기 앉아라. \n선생님이 맨 앞에 자리를 손으로 가리쳤습니다. \n빈자리였습니다.',
 '영택이는 비틀비틀 걸있습니다. \n목발을 바닥에 놓았습니다. \n의자에 앉았습니다.',
 '아주머니와 선생님은 복도에서 대화를 했습니다.',
 '석우: 야, 영택이가 1 학년 때 5 반이었지? \n석우가 서경이에게 물었습니다. \n서경이는 석우와 이웃 동네에 사는 친구입니다. \n서경: 응, 영택이는 공부를 잘한대. 하지만 영택이는 친구가 없대.',
 '선생님이 들어왔습니다. \n아이들을 보면서 말했습니다. \n\n선생님: 집이 제일교회랑 가까운 사람이 누구니? \n\n제일교회는 가장 큰 교회입니다. \n석우 집에서 가깝습니다. \n\n석우: 저요. \n\n석우는 조심스럽게 손을 들있습니다. \n\n선생님: 너 이름이 뭐니? \n\n석우: 제 이름은 문석우입니다. \n\n선생님: 학교 오는 길에 새한마트 있지? \n\n석우: 네. \n\n새한마트는 가장 큰 슈퍼마켓입니다. \n\n선생님: 영택이의 집은 새한마트 옆이다. 내일부터 석우는 학교 올 때 영택이 집에 간다. 영택이 가방을 들어 주어라. 같이 학교에 오너라. \n\n석우는 대답하지 않았습니다.',
 '석우는 놀랐습니다.',
 '선생님: 2 학년 동안만 고생해라. \n선생님이 무섭게 말했습니다. \n석우: 네. \n석우는 작게 대답했습니다.',
 '선생님은 1 학기 시간표와 주의사항을 말했습니다. \n주의사항은 조심해야 하는 것입니다. \n오늘 수업은 끝났습니다. \n선생님이 교실에서 나갔습니다.',
 '서경이가 석우에게 말했습니다.\n서경: 석우야, 너 어떡하니?\n석우: 할 수 없지.\n석우는 선생님이 생각났습니다.\n석우는 선생님이 무서웠습니다. \n석우는 영택이에게 다가갔습니다. \n석우가 영택이에게 말했습니다.',
 '석우: 영택아. 나에게 가방을 줘.\n영택이는 석우의 눈을 보지 못했습니다.\n영택이는 석우에게 가방을 주었습니다.',
]

In [309]:
df = pd.DataFrame()
df['user'] = a + c
df['reply'] = b + d
df

,user,reply
0,<개학 날 맡은 업무>,<개학 날 내가 맡은 일>
1,오늘은 2학년이 되는 새 학기 첫날입니다.,오늘은 2 학년 첫날입니다.
2,드르륵 문이 열리는 소리가 나더니 2학년 2반 교실 안으로 누군가 들어섰습니다. 조...,문이 열리고 2 학년 2 반 교실에 누군가 들어왔습니다. \n조기준 담임 선생님과 ...
3,잿빛 코트를 입은 아주머니 한 분이 조심스럽게 고개를 숙여 인사했습니다.,아주머니는 회색 외투를 입었습니다. \n아주머니가 고개를 숙여 인사했습니다.
4,"“선생님, 안녕하세요? 이 반에 배정된 민영택이 엄마입니다.”","아주머니: 선생님, 안녕하세요. 저는 민영택 엄마입니다."
5,"“아, 그러세요? 그런데 영택이는요?”",선생님: 영택이는 어디 있나요?
6,“여기 있습니다.”,아주머니: 뒤에 있습니다.
7,아주머니 뒤로 영택이라는 아이가 들어왔습니다. 양 팔꿈치에 하얀 알루미늄으로 된 목...,영택이가 들어왔습니다. \n영택이는 걷기 어려운 학생입니다. \n영택이는 양 팔꿈치...
8,“여기 앉아라.” 선생님이 비어 있는 맨 앞자리를 가리켰습니다.,선생님: 여기 앉아라. \n선생님이 맨 앞에 자리를 손으로 가리쳤습니다. \n빈자리...
9,영택이는 비틀거리며 걸어와 목발을 바닥에 내려 놓고 걸상에 털썩 앉았습니다.,영택이는 비틀비틀 걸있습니다. \n목발을 바닥에 놓았습니다. \n의자에 앉았습니다.


In [308]:
c = [
 '다른 사람이 들어 줘야 하기 때문인지 영택이 가방은 다른 아이들 것처럼 배낭 모양이 아니라 긴 끈이 하나만 달려 있었습니다.',
 '석우가 어색한 것처럼 영택이도 석우에게 신세를 지는 것이 미안한 표정이었습니다.',
 '석우는 아무 말 없이 가방을 어깨에 메고 교실을 나섰습니다. 오늘은 새 학기 첫날이라 가방이 가볍습니다. 하지만 내일부터 시간표대로 책을 담아 오면 조금은 무거울 겁니다.',
 '“에이, 먹는 조기는 맛있기나 하지. 우리 조기는 쓸데없는 것만 시키고 난리야.”',
 '조기는 조기준 선생님 별명입니다.',
 '석우는 투덜대며 길가에 있는 돌멩이를 걷어찼습니다. 하루도 빠지지 않고 남의 가방을 들어다 줘야 한다니…….',
 '어쩌다 재수 없게 이 일을 맡았는지 모릅니다. 아까 제일교회 근처에 사는 사람 손들라고 할 때 들지 말아야 했습니다.',
 '교문에서 뒤돌아보니 영택이는 그제야 학교 건물을 빠져나와 운동장을 걷고 있었습니다. 그 걸음걸이로는 거북이나 달팽이와 경주해도 질 것 같았습니다.',
 '석우를 본 영택이가 힘이 없어 잘 들리지도 않는 소리로 외쳤습니다. “먼저 가! 나, 난 한참 걸려.” 석우는 잠시 머뭇거렸습니다.',]

In [307]:
d = ['영택이의 가방은 끈이 하나입니다. \n영택이의 가방은 다른 사람이 들어 줍니다. \n석우는 영택이의 가방을 들어 줍니다.',
 '그래서 영택이는 석우에게 미안합니다.\n석우는 영택이의 가방을 들었습니다.',
 '석우는 교실을 나갔습니다.\n오늘은 2 학년 첫날입니다.\n책이 없어서 가방이 가볍습니다.\n내일부터는 책을 가져오면 가방이 무거울 것입니다.',
 '석우가 말했습니다.\n석우: 먹는 조기는 맛있지. 우리 조기는 쓸데없는 것만 시키는 거야. \n먹는 조기는 물고기입니다.',
 '우리 조기는 조기준 선생님의 별명입니다.',
 '석우는 길가에 있는 돌멩이를 찼습니다.\n석우: 매일 영택이의 가방을 들어 줘야 한다니……,',
 '석우는 재수 없다고 생각했습니다.\n재수는 좋은 일이 생기는 행운을 말합니다.\n석우는 손을 들지 말아야 했다고 생각했습니다.',
 '석우는 교문에서 뒤를 봤습니다.\n영택이는 운동장을 걷고 있습니다.\n영택이는 거북이와 달팽이와 경주해도 질 것 같습니다.',
 '영택이는 석우를 보았습니다.\n영택이는 석우에게 말했습니다.\n영택: 먼저 가! 난 한참 걸려. \n석우는 잠시 생각했습니다.',]

In [ ]:
# print the first 5 lines of the training file
!head -n 5 finetune_training.jsonl

### Upload files


In [218]:
training_file_name =  './data/exdata4.jsonl'
validation_file_name = './data/validation_jsl21.jsonl'


In [ ]:
with open(training_file_name, "rb") as training_fd:
    training_response = client.files.create(
        file=training_fd, purpose="fine-tune"
    )

training_file_id = training_response.id

with open(validation_file_name, "rb") as validation_fd:
    validation_response = client.files.create(
        file=validation_fd, purpose="fine-tune"
    )
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

## Function calling

In [ ]:
functions = [
    {
        "name": "return_index",
        "description": "입력된 글을 바탕으로 목차를 만들어줘.",
        "parameters": {
            "type": "object",
            "properties": {
                "ingredients": {
                    "type": "string",
                    "description": "The ingredients list."
                },
                "steps": {
                    "type": "string",
                    "description": "The recipe steps."
                },
            },
            },
            "required": ["ingredients","steps"],
        }
]

recipe = 'Fish and chips'
query = f"What is the recipe for {recipe}? Return the ingredients list and steps separately."


response = openai.ChatCompletion.create(
    model= fine_tuned_model_id,
    messages=[{"role": "user", "content": query}],
    functions=functions,
    function_call={'name':'return_recipe'}
)
response_message = response["choices"][0]["message"]
print(response_message)
print(response_message['function_call']['arguments'])

## Fine-tuning


In [324]:
x_model_name = 'gpt-3.5-turbo-0613'

In [220]:
x_model_name = fine_tuned_model_id

In [ ]:
fine_tuned_model_id


In [ ]:
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    
    model=x_model_name,
    hyperparameters={
            "n_epochs": 15,
        # "prompt_loss_weight": 0.1,
        # "compute_classification_metrics": True,
        "batch_size": 32
              }
    # suffix="easy-read",
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

In [ ]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)


### Training Loss (훈련 손실)
- 정의: 모델이 훈련 데이터에 대해 예측한 값과 실제 값 간의 오차를 나타냅니다. 훈련 과정에서 모델이 얼마나 잘 학습하고 있는지를 나타내는 지표입니다.
- 의미: 훈련 손실이 낮을수록 모델이 훈련 데이터에 잘 맞춰져 있다는 것을 의미합니다. 하지만 훈련 손실만으로는 모델의 일반화 능력을 판단하기 어렵습니다.
### Validation Loss (검증 손실)
- 정의: 모델이 검증 데이터에 대해 예측한 값과 실제 값 간의 오차를 나타냅니다. 검증 데이터는 모델이 훈련 중에 보지 않은 데이터로, 모델의 일반화 능력을 평가하는 데 사용됩니다.
- 의미: 검증 손실이 낮을수록 모델이 새로운 데이터에 대해서도 잘 일반화되고 있다는 것을 의미합니다. 훈련 손실과 비교하여 검증 손실이 높아지면 과적합(overfitting)의 가능성이 있습니다.
#### 요약
훈련 손실은 모델이 훈련 데이터에 얼마나 잘 맞는지를 보여주고, 검증 손실은 모델이 새로운 데이터에 대해 얼마나 잘 일반화되는지를 보여줍니다.
훈련 손실이 낮더라도 검증 손실이 높다면, 모델이 훈련 데이터에 과적합되었을 가능성이 있으므로 주의해야 합니다.
이 두 손실을 함께 모니터링하면 모델의 성능을 보다 효과적으로 평가하고 조정할 수 있습니다.

Step 48/63: training loss=0.14, validation loss=1.20, full validation loss=1.20
Step 49/63: training loss=0.08, validation loss=1.20
Step 50/63: training loss=0.06, validation loss=1.22
Step 51/63: training loss=0.08, validation loss=1.22
Step 52/63: training loss=0.04, validation loss=1.22, full validation loss=1.22
Step 53/63: training loss=0.06, validation loss=1.24
Step 54/63: training loss=0.13, validation loss=1.23
Step 55/63: training loss=0.14, validation loss=1.23
Step 56/63: training loss=0.06, validation loss=1.25, full validation loss=1.24
Step 57/63: training loss=0.05, validation loss=1.24
Step 58/63: training loss=0.03, validation loss=1.24
Step 59/63: training loss=0.04, validation loss=1.25
Step 60/63: training loss=0.03, validation loss=1.24, full validation loss=1.24
Step 61/63: training loss=0.06, validation loss=1.24
Step 62/63: training loss=0.13, validation loss=1.26
Step 63/63: training loss=0.05, validation loss=1.24

In [ ]:
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Now that it's done, we can get a fine-tuned model ID from the job:


In [ ]:
response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None: 
    raise RuntimeError("Fine-tuned model ID not found. Your job has likely not been completed yet.")

print("Fine-tuned model ID:", fine_tuned_model_id)

## 모델 테스트

user_message에 모델에 넣을 글을 작성하면, 학습 된 모델이 글을 바꿔줌


In [341]:
user_message = '판매자 대금 정산 지연 사태가 소비자 피해까지 확산한 티몬이 결국 이날 오전부터 강남구 본사에서 수기로 환불 접수를 시작했다. 수백 명의 피해자가 강남구 본사 건물을 찾아 거세게 항의를 이어가자 금일 새벽 2시경부터 뒤늦은 대처에 나선 것. 하지만 아직 계좌 송금 처리는 진행되지 않은 것으로 알려졌다.'

In [334]:
user_message = """
그렇긴 하지만, 모든 유형의 입력이 유용한 출력을 생성하는 것은 아닙니다. 생성형 AI 시스템이 정확하고 연관성 있는 응답을 생성하기 위해서는 컨텍스트와 세부 정보가 필요합니다. 프롬프트를 체계적으로 설계하면 보다 의미 있고 유용한 결과물을 얻을 수 있습니다. 프롬프트 엔지니어링에서는 AI 시스템에서 원하는 결과를 얻을 때까지 프롬프트를 지속적으로 조정합니다.
"""


In [338]:
user_message = """
엘리베이터 내에 부착된 유의사항을 지켜야 합니다.
탑승정원이나 적재하중의 초과는 고장과 사고의 원인이 될 수 있습니다.
조작반의 인터폰, 비상정지스위치 등을 장난으로 조작하지 맙시다.
버튼을 불필요하게 누르거나 난폭하게 다루지 맙시다.
운행 중인 엘리베이터 내에서 뛰거나 심한 장난을 하지 맙시다.
엘리베이터 출입문을 흔들거나 손으로 밀지 말아야 하며 출입문에 기대지 맙시다.
엘리베이터 내에서는 담배를 피우지 맙시다.
어린이나 노약자는 될 수 있으면 보호자와 함께 탑승하도록 하고 애완동물은 안고 탑승합시다.
지정된 용도 이외에는 사용하지 맙시다.
출입문의 문턱 틈이나 문 사이에 이물질을 버리지 맙시다.
관리자의 입회·허락 없이 크거나 무거운 화물을 무단으로 싣지 맙시다.
"""

In [342]:
user_message = "금일 도서관 휴관입니다."

In [343]:
# test_df = recipe_df.loc[201:300]
# test_row = test_df.iloc[0]
test_messages = []
test_messages.append({"role": "system", "content": system_message})
# user_message = create_user_message(test_row)
test_messages.append({"role": "user", "content": user_message})

# pprint(test_messages)
test_messages

[{'role': 'system',
  'content': '너는 입력받은 한국어를 쉬운 한국어로 변환해주는 도우미야. 다음 조건들을 모두 충족하는 내용으로 변환해서 알려줘. 1. 입력받은 한국어 문장을 이해하기 쉬운 한국어 문장으로 변환해줘. 2. 문장은 간결하게, 한 문장이 길어지면 두 문장으로 나눠서 변환해줘. 3. 꾸미는 말 빼고, 이어진 문장은 두 개의 문장으로 변환해줘. 4. 주어를 중심으로 알기 쉽게 변환해줘. 5. 최대한 능동형 문장으로, 서술식의 구어체(-합니다, -입니다)로 변환해줘. 6. 추상적 표현과 비유는 자제하도록 해. 7. 이중부정 문장은 이해하기 쉬운 문장으로 바꿔. 8. 한 문장에 한 줄씩 적어야해. 9. 대화문은 문장 전후로 한 줄 띄어줘. 10. 단어는 일상생활에서 자주 쓰는, 가능한 짧고 이해하기 쉬운 단어로 사용하도록 해. 11. 한자어나 외국어를 풀어서 쉬운 말로 변환해. 12. 약어가 있으면 다음 문장에 설명을 추가해.'},
 {'role': 'user', 'content': '금일 도서관 휴관입니다.'}]

In [344]:
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=1000
)
print(response.choices[0].message.content)

오늘 도서관은 쉽니다.


In [227]:
def get_usage(api_key, date, use_own_key=False):
    headers = {'Authorization': f'Bearer {api_key}'}
    params = {'date': date}

    print(f"use_own_key: {use_own_key}")
    if use_own_key:
        try:
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()  # Raises a HTTPError if the response was unsuccessful
            response = response.json()
        except requests.RequestException as e:
            print(f"Request to OpenAI API failed: {e}")
            return None, None
    else:
        """
        To avoid "Request to OpenAI API failed: 429 Client Error: Too Many Requests for url: https://api.openai.com/v1/usage"
        """
        with open('static/data.json', 'r') as f:
            response = json.load(f)
    try:
        usage_data = response['data']
        whisper_data = response['whisper_api_data']
    except KeyError as e:
        print(f"Error processing OpenAI API response: {e}")
        return None, None

    return usage_data, whisper_data
    